In [3]:
# Import dependencies
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager




/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_36916/4240391624.py:20: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(


In [6]:

# Scrape OurLads site to see how highly ranked players are on the rosters of their NFL teams
def scrape_depthcharts():
    # Set Selenium/Chrome settings
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.binary_location = os.environ.get("GOOGLE_CHROME_BIN") # Enable this in deployed version
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    capa = DesiredCapabilities.CHROME
    capa["pageLoadStrategy"] = "none"
    driver = webdriver.Chrome(
        # executable_path=os.environ.get("CHROMEDRIVER_PATH"), # Enable this in deployed version
        service=Service("/Applications/chromedriver"), # Enable this when running locally
        chrome_options=chrome_options, 
        desired_capabilities=capa)
    # Tell Selenium to wait until the page has time to fully load
    wait = WebDriverWait(driver, 20)

    # Scrape OurLads website
    url = f"https://www.ourlads.com/nfldepthcharts/depthcharts.aspx"
    # Use Selenium Chromedriver to read OurLads scouting website
    driver.get(url)
    # Tell Selenium the item that we want to watch for to see it fully load, then stop running
    wait.until(EC.presence_of_element_located((By.XPATH, "//table[@id='ctl00_phContent_gvChart']")))
    driver.execute_script("window.stop();")

    # Put the scraped html table into a dataframe
    df = pd.read_html(
        driver.find_element(By.XPATH, value="//table[@id='ctl00_phContent_gvChart']").get_attribute("outerHTML")
        )[0]
    # Filter for only the needed columns
    df = df.loc[:, ['Team', 'Pos', 'Player 1', 'Player 2', 'Player 3', 'Player 4', 'Player 5']]
    # Filter only relevant positions
    posList = ['LWR', 'RWR', 'SWR', 'TE', 'QB', 'RB', 'PK', 'PR', 'KR', 'RES']
    df = df.loc[df['Pos'].isin(posList)]
    # Transpose columns to rows to get position ranks in row form rather than column
    df = df.melt(id_vars=["Team", "Pos"], 
        var_name="posRank", 
        value_name="playerName")
        # Rename columns to match MyFantasyLeague
    df = df.rename(columns={'Team':'team', 'Pos':'pos'})
    # Clean the Team column to match My Fantasy League abbreviations
    teamDict = {
    'ARZ':'ARI', 'ATL':'ATL', 'BAL':'BAL', 'BUF':'BUF', 'CAR':'CAR', 'CHI':'CHI', 'CIN':'CIN', 'CLE':'CLE', 
    'DAL':'DAL', 'DEN':'DEN', 'DET':'DET', 'GB':'GBP', 'HOU':'HOU', 'IND':'IND', 'JAX':'JAC', 'KC':'KCC', 
    'LAC':'LAC', 'LAR':'LAR', 'LV':'LVR', 'MIA':'MIA', 'MIN':'MIN', 'NE':'NEP', 'NO':'NOS', 'NYG':'NYG', 
    'NYJ':'NYJ', 'PHI':'PHI', 'PIT':'PIT', 'SEA':'SEA', 'SF':'SFO', 'TB':'TBB', 'TEN':'TEN', 'WAS':'WAS'
    }
    df['team'] = df['team'].map(teamDict)
    # Clean the Name column    
    
    # Create a playerID column
    df['id_ol'] = df['team'] + df['playerName']
    # Clean the Position column
    # Generalize wide receiver specific positions to WR
    df['pos'].replace(["LWR", "RWR", "SWR"], "WR", inplace=True)
    # Identify punt returners and kick returners, then drop those rows from the df
    prs = df.loc[df['pos']=='PR']
    krs = df.loc[df['pos']=='KR']
    df.loc[df['id_ol'].isin(prs['id_ol']), 'PR'] = "YES"
    df.loc[df['id_ol'].isin(krs['id_ol']), 'KR'] = "YES"
    df = df.loc[(df['pos']!="PR") & (df['pos']!="PR")]
    df['PR'].fillna("NO", inplace=True)
    df['KR'].fillna("NO", inplace=True)
    # Clean the posRank column
    df['posRank'] = .map
    df['posRank'] = df['Pos'] + df['posRank']

    return df
scrape_depthcharts()

/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_36916/3530790968.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(


,Team,Pos,posRank,playerName
0,ARZ,LWR,Player 1,"HOPKINS, DEANDRE T/Hou"
1,ARZ,RWR,Player 1,"Moore, Rondale 21/2"
2,ARZ,SWR,Player 1,"Dortch, Greg SF21"
3,ARZ,TE,Player 1,"ERTZ, ZACH T/Phi"
4,ARZ,QB,Player 1,"Murray, Kyler 19/1"
...,...,...,...,...
1780,WAS,PR,Player 5,NaN
1781,WAS,KR,Player 5,NaN
1782,WAS,RES,Player 5,"White, Drew IR"
1783,WAS,RES,Player 5,"Eifler, Milo IR-5"
